In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies= pd.read_csv('ml-1m/movies.dat',sep = '::', header = None, engine = 'python', encoding='latin-1')

In [3]:
movies[movies[1] == 'Toy Story (1995)']

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy


In [4]:
users= pd.read_csv('ml-1m/users.dat',sep = '::', header = None, engine = 'python', encoding='latin-1')

In [5]:
users

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [6]:
ratings= pd.read_csv('ml-1m/ratings.dat',sep = '::', header = None, engine = 'python', encoding='latin-1')

In [7]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


column 0 is the user id <br>
column 1 is the movie id <br>
column 2 is the rating given by the user <br>
column 3 is the time stamp <br>
Note: For the user and movie details we can refer their data set loaded above

### Preparing the data

In [8]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')

In [9]:
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [10]:
training_set= np.array(training_set, dtype = 'int')

In [22]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [23]:
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')

In [24]:
test_set= np.array(test_set, dtype = 'int')

#### Getting the number of movies and number of users

In [25]:
nb_users=  int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_users

943

In [26]:
nb_movies=  int(max(max(training_set[:,1]), max(test_set[:,1])))
nb_movies

1682

#### Function for converting a matrix of ratings as per user. rows has user and columns has movie wise ratings
$$\begin{bmatrix} 1 & 2 & 1 & ....\\ 3 & 0 & 1 & ....\\ 0 & 2 & 4 & .... \\ . \\ .\end{bmatrix}$$ <br>
where 0 rating means user did not rate the movie

In [29]:
def convert(data):
    new_data = []
    for user in range(1,nb_users+1):
        id_movies= data[:,1][data[:,0]==user]
        id_rating = data[:,2][data[:,0]==user]
        rating= np.zeros(nb_movies)
        rating[id_movies-1] = id_rating
        new_data.append(list(rating))
    return new_data
        

In [30]:
training_set= convert(training_set)

In [31]:
test_set= convert(test_set)

In [32]:
len(training_set),len(training_set[0]) # dimenssion of the data

(943, 1682)

### Converting data into tensors

In [33]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

### Autoencoder architecture 
#### Stacked auto encoder

In [44]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20) # out feature =20 is just an experimental value. We could tune it for best
        self.fc2 = nn.Linear(20, 15) # 2nd hidden layer
        self.fc3 = nn.Linear(15, 20) # 3rd hidden layer (Decoding or decompression process)
        self.fc4 = nn.Linear(20, nb_movies) # 2nd hidden layer
        self.activation = nn.Hardtanh()
        self.activation1 = nn.Softmax()
    def forward(self, x):
        x = self.activation(self.fc1(x)) # x in fc1(x) is the first input vector applied to the network
        x = self.activation(self.fc2(x)) # x in fc2(x) is the outpout of first hidden layer
        x= self.activation1(self.fc3(x))
        x = self.fc4(x) # now x will be the output of the Auto encoder
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay=0.5)

### Training SAE

In [45]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.  # we will use it to exclude the users who havent rate any movie
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0) # to add a new dimession as per teh reqirements of nn
        target = input.clone()
        if torch.sum(target.data>0) > 0:
            output = sae(input)
            target.required_grad = False
            output[target==0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data>0) +1e-10) # 1e-10 to avoid devison by zero
            loss.backward() # decide the dirction in which weights are updated
            train_loss += np.sqrt(loss.data* mean_corrector)
            s += 1.
            optimizer.step()  # decides the amount by what weights must be updated
    print('epoch: ' + str(epoch) + '   loss: '+ str(train_loss/s))

C:\Users\katiy\anaconda3\envs\gpu_test\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


epoch: 1   loss: tensor(3.2419)
epoch: 2   loss: tensor(2.4788)
epoch: 3   loss: tensor(2.0611)
epoch: 4   loss: tensor(1.8061)
epoch: 5   loss: tensor(1.6391)
epoch: 6   loss: tensor(1.5222)
epoch: 7   loss: tensor(1.4376)
epoch: 8   loss: tensor(1.3731)
epoch: 9   loss: tensor(1.3236)
epoch: 10   loss: tensor(1.2836)
epoch: 11   loss: tensor(1.2517)
epoch: 12   loss: tensor(1.2248)
epoch: 13   loss: tensor(1.2030)
epoch: 14   loss: tensor(1.1839)
epoch: 15   loss: tensor(1.1681)
epoch: 16   loss: tensor(1.1540)
epoch: 17   loss: tensor(1.1422)
epoch: 18   loss: tensor(1.1314)
epoch: 19   loss: tensor(1.1224)
epoch: 20   loss: tensor(1.1139)
epoch: 21   loss: tensor(1.1068)
epoch: 22   loss: tensor(1.0998)
epoch: 23   loss: tensor(1.0942)
epoch: 24   loss: tensor(1.0885)
epoch: 25   loss: tensor(1.0839)
epoch: 26   loss: tensor(1.0792)
epoch: 27   loss: tensor(1.0753)
epoch: 28   loss: tensor(1.0713)
epoch: 29   loss: tensor(1.0681)
epoch: 30   loss: tensor(1.0646)
epoch: 31   loss: t

### Testing the AutoEncoder

In [77]:
x = [1,2,3,4,5,0,0]
x = torch.FloatTensor(x)
input = Variable(x).unsqueeze(0)
input.require_grad = False

In [80]:
test_loss = 0
s = 0.
for id_users in range(nb_users):
    input = Variable(training_set[id_users]).unsqueeze(0) # here we taken training set because SAE will take the ratings 
    ## of the user who has given the rating to the movie he has watched. Based on that data we will prdict that what raing
    ## user will give to those movies which were not watched by particular user.
    target = Variable(test_set[id_users]).unsqueeze(0) # target contains actual ratings
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target ==0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))
            
        

test loss: tensor(0.9914)


C:\Users\katiy\anaconda3\envs\gpu_test\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
